In [13]:
%reload_ext autoreload
%autoreload 2
import os
import glob
import scipy.io.wavfile as wv
import matplotlib.pyplot as plt
import numpy as np
from util import util
import scipy.signal as sig

import numpy
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

In [2]:
nrFiles = 2
Y_train = np.array((np.ones(nrFiles),np.zeros(nrFiles), np.zeros(nrFiles))).T

nrFiles = 4
Y_train_nw = np.array((np.zeros(nrFiles),np.ones(nrFiles), np.zeros(nrFiles))).T
Y_train = np.concatenate((Y_train, Y_train_nw),axis=0)

nrFiles = 3
Y_train_nw = np.array((np.zeros(nrFiles),np.zeros(nrFiles), np.ones(nrFiles))).T
Y_train = np.concatenate((Y_train, Y_train_nw),axis=0)
print(np.shape(Y_train))
print(Y_train)

(9, 3)
[[ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]]


In [3]:
import pandas

# load dataset
dataframe = pandas.read_csv("dummy/iris.csv", header=None)
dataset = dataframe.values
X = dataset[1:,0:4].astype(float)
Y = dataset[1:,4]


In [4]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)


In [5]:
encoded_Y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2], dtype=int64)

In [6]:
encoder

LabelEncoder()

In [7]:
Y

array(['setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa',
       'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa',
       'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa',
       'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa',
       'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa',
       'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa',
       'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa',
       'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa',
       'setosa', 'setosa', 'versicolor', 'versicolor', 'versicolor',
       'versicolor', 'versicolor', 'versicolor', 'versicolor',
       'versicolor', 'versicolor', 'versicolor', 'versicolor',
       'versicolor', 'versicolor', 'versicolor', 'versicolor',
       'versicolor', 'versicolor', 'versicolor', 'versicolor',
       'versicolor', 'versicolor', 'versicolor', 'versicolor',
       'versicolor', 'versicolor', 'versicolor', 'versicolor',
       'versicolo

In [8]:
dummy_y

array([[ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.

In [9]:
dataframe

,0,1,2,3,4
0,sepal_length,sepal_width,petal_length,petal_width,species
1,5.1,3.5,1.4,0.2,setosa
2,4.9,3,1.4,0.2,setosa
3,4.7,3.2,1.3,0.2,setosa
4,4.6,3.1,1.5,0.2,setosa
5,5,3.6,1.4,0.2,setosa
6,5.4,3.9,1.7,0.4,setosa
7,4.6,3.4,1.4,0.3,setosa
8,5,3.4,1.5,0.2,setosa
9,4.4,2.9,1.4,0.2,setosa


In [10]:
X

array([[ 5.1,  3.5,  1.4,  0.2],
       [ 4.9,  3. ,  1.4,  0.2],
       [ 4.7,  3.2,  1.3,  0.2],
       [ 4.6,  3.1,  1.5,  0.2],
       [ 5. ,  3.6,  1.4,  0.2],
       [ 5.4,  3.9,  1.7,  0.4],
       [ 4.6,  3.4,  1.4,  0.3],
       [ 5. ,  3.4,  1.5,  0.2],
       [ 4.4,  2.9,  1.4,  0.2],
       [ 4.9,  3.1,  1.5,  0.1],
       [ 5.4,  3.7,  1.5,  0.2],
       [ 4.8,  3.4,  1.6,  0.2],
       [ 4.8,  3. ,  1.4,  0.1],
       [ 4.3,  3. ,  1.1,  0.1],
       [ 5.8,  4. ,  1.2,  0.2],
       [ 5.7,  4.4,  1.5,  0.4],
       [ 5.4,  3.9,  1.3,  0.4],
       [ 5.1,  3.5,  1.4,  0.3],
       [ 5.7,  3.8,  1.7,  0.3],
       [ 5.1,  3.8,  1.5,  0.3],
       [ 5.4,  3.4,  1.7,  0.2],
       [ 5.1,  3.7,  1.5,  0.4],
       [ 4.6,  3.6,  1. ,  0.2],
       [ 5.1,  3.3,  1.7,  0.5],
       [ 4.8,  3.4,  1.9,  0.2],
       [ 5. ,  3. ,  1.6,  0.2],
       [ 5. ,  3.4,  1.6,  0.4],
       [ 5.2,  3.5,  1.5,  0.2],
       [ 5.2,  3.4,  1.4,  0.2],
       [ 4.7,  3.2,  1.6,  0.2],
       [ 4

In [11]:
# define baseline model
def baseline_model():
	# create model
	model = Sequential()
	model.add(Dense(8, input_dim=4, activation='relu'))
	model.add(Dense(3, activation='softmax'))
	# Compile model
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model

In [19]:
estimator = KerasClassifier(build_fn=baseline_model, epochs=200, batch_size=5, verbose=1)

In [20]:
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)
kfold = KFold(n_splits=10, shuffle=True, random_state=seed)

In [21]:
results = cross_val_score(estimator, X, dummy_y, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Epoch 1/200
135/135 [==============================] - 0s 3ms/step - loss: 1.2226 - acc: 0.3333
Epoch 2/200
135/135 [==============================] - 0s 829us/step - loss: 1.1148 - acc: 0.4741
Epoch 3/200
135/135 [==============================] - 0s 658us/step - loss: 1.0349 - acc: 0.6815
Epoch 4/200
135/135 [==============================] - 0s 607us/step - loss: 0.9960 - acc: 0.6815
Epoch 5/200
135/135 [==============================] - 0s 614us/step - loss: 0.9656 - acc: 0.6815
Epoch 6/200
135/135 [==============================] - 0s 614us/step - loss: 0.9354 - acc: 0.6815
Epoch 7/200
135/135 [==============================] - 0s 799us/step - loss: 0.9070 - acc: 0.6815
Epoch 8/200
135/135 [==============================] - 0s 636us/step - loss: 0.8783 - acc: 0.6815
Epoch 9/200
135/135 [==============================] - 0s 585us/step - loss: 0.8505 - acc: 0.6815
Epoch 10/200
135/135 [==============================] - 0s 570us/step - loss: 0.8244 - acc: 0.6815
Epoch 11/200
135/135 

135/135 [==============================] - 0s 688us/step - loss: 0.2463 - acc: 0.9704
Epoch 84/200
135/135 [==============================] - 0s 681us/step - loss: 0.2425 - acc: 0.9630
Epoch 85/200
135/135 [==============================] - 0s 570us/step - loss: 0.2391 - acc: 0.9704
Epoch 86/200
135/135 [==============================] - 0s 562us/step - loss: 0.2364 - acc: 0.9704
Epoch 87/200
135/135 [==============================] - 0s 540us/step - loss: 0.2338 - acc: 0.9704
Epoch 88/200
135/135 [==============================] - 0s 555us/step - loss: 0.2324 - acc: 0.9704
Epoch 89/200
135/135 [==============================] - 0s 555us/step - loss: 0.2291 - acc: 0.9704
Epoch 90/200
135/135 [==============================] - 0s 548us/step - loss: 0.2261 - acc: 0.9704
Epoch 91/200
135/135 [==============================] - 0s 555us/step - loss: 0.2252 - acc: 0.9704
Epoch 92/200
135/135 [==============================] - 0s 562us/step - loss: 0.2208 - acc: 0.9704
Epoch 93/200
135/135 [=

135/135 [==============================] - 0s 548us/step - loss: 0.1178 - acc: 0.9778
Epoch 166/200
135/135 [==============================] - 0s 562us/step - loss: 0.1181 - acc: 0.9704
Epoch 167/200
135/135 [==============================] - 0s 577us/step - loss: 0.1161 - acc: 0.9778
Epoch 168/200
135/135 [==============================] - 0s 548us/step - loss: 0.1168 - acc: 0.9778
Epoch 169/200
135/135 [==============================] - 0s 555us/step - loss: 0.1145 - acc: 0.9778
Epoch 170/200
135/135 [==============================] - 0s 548us/step - loss: 0.1156 - acc: 0.9778
Epoch 171/200
135/135 [==============================] - 0s 533us/step - loss: 0.1142 - acc: 0.9778
Epoch 172/200
135/135 [==============================] - 0s 540us/step - loss: 0.1148 - acc: 0.9704
Epoch 173/200
135/135 [==============================] - 0s 570us/step - loss: 0.1175 - acc: 0.9704
Epoch 174/200
135/135 [==============================] - 0s 562us/step - loss: 0.1130 - acc: 0.9778
Epoch 175/200


135/135 [==============================] - 0s 577us/step - loss: 0.2182 - acc: 0.9778
Epoch 48/200
135/135 [==============================] - 0s 555us/step - loss: 0.2154 - acc: 0.9778
Epoch 49/200
135/135 [==============================] - 0s 562us/step - loss: 0.2063 - acc: 0.9778
Epoch 50/200
135/135 [==============================] - 0s 555us/step - loss: 0.2026 - acc: 0.9778
Epoch 51/200
135/135 [==============================] - 0s 644us/step - loss: 0.1992 - acc: 0.9778
Epoch 52/200
135/135 [==============================] - 0s 622us/step - loss: 0.1965 - acc: 0.9778
Epoch 53/200
135/135 [==============================] - 0s 614us/step - loss: 0.1962 - acc: 0.9778
Epoch 54/200
135/135 [==============================] - 0s 651us/step - loss: 0.1922 - acc: 0.9704
Epoch 55/200
135/135 [==============================] - 0s 725us/step - loss: 0.1865 - acc: 0.9778
Epoch 56/200
135/135 [==============================] - 0s 548us/step - loss: 0.1856 - acc: 0.9778
Epoch 57/200
135/135 [=

135/135 [==============================] - 0s 548us/step - loss: 0.0906 - acc: 0.9704
Epoch 130/200
135/135 [==============================] - 0s 562us/step - loss: 0.0889 - acc: 0.9778
Epoch 131/200
135/135 [==============================] - 0s 555us/step - loss: 0.0876 - acc: 0.9778
Epoch 132/200
135/135 [==============================] - 0s 614us/step - loss: 0.0896 - acc: 0.9704
Epoch 133/200
135/135 [==============================] - 0s 644us/step - loss: 0.0887 - acc: 0.9778
Epoch 134/200
135/135 [==============================] - 0s 570us/step - loss: 0.0894 - acc: 0.9704
Epoch 135/200
135/135 [==============================] - 0s 562us/step - loss: 0.0879 - acc: 0.9778
Epoch 136/200
135/135 [==============================] - 0s 570us/step - loss: 0.0859 - acc: 0.9778
Epoch 137/200
135/135 [==============================] - 0s 548us/step - loss: 0.0843 - acc: 0.9778
Epoch 138/200
135/135 [==============================] - 0s 533us/step - loss: 0.0838 - acc: 0.9778
Epoch 139/200


...

135/135 [==============================] - 0s 548us/step - loss: 0.3426 - acc: 0.9556
Epoch 39/200
135/135 [==============================] - 0s 548us/step - loss: 0.3353 - acc: 0.9481
Epoch 40/200
135/135 [==============================] - 0s 570us/step - loss: 0.3280 - acc: 0.9704
Epoch 41/200
135/135 [==============================] - 0s 592us/step - loss: 0.3223 - acc: 0.9481
Epoch 42/200
135/135 [==============================] - 0s 614us/step - loss: 0.3170 - acc: 0.9556
Epoch 43/200
135/135 [==============================] - 0s 651us/step - loss: 0.3089 - acc: 0.9481
Epoch 44/200
135/135 [==============================] - 0s 585us/step - loss: 0.3026 - acc: 0.9778
Epoch 45/200
135/135 [==============================] - 0s 577us/step - loss: 0.2989 - acc: 0.9481
Epoch 46/200
135/135 [==============================] - 0s 555us/step - loss: 0.2923 - acc: 0.9704
Epoch 47/200
135/135 [==============================] - 0s 592us/step - loss: 0.2841 - acc: 0.9778
Epoch 48/200
135/135 [=

135/135 [==============================] - 0s 540us/step - loss: 0.1091 - acc: 0.9778
Epoch 121/200
135/135 [==============================] - 0s 562us/step - loss: 0.1108 - acc: 0.9704
Epoch 122/200
135/135 [==============================] - 0s 555us/step - loss: 0.1101 - acc: 0.9778
Epoch 123/200
135/135 [==============================] - 0s 555us/step - loss: 0.1100 - acc: 0.9778
Epoch 124/200
135/135 [==============================] - 0s 548us/step - loss: 0.1064 - acc: 0.9778
Epoch 125/200
135/135 [==============================] - 0s 622us/step - loss: 0.1068 - acc: 0.9778
Epoch 126/200
135/135 [==============================] - 0s 696us/step - loss: 0.1061 - acc: 0.9778
Epoch 127/200
135/135 [==============================] - 0s 555us/step - loss: 0.1076 - acc: 0.9704
Epoch 128/200
135/135 [==============================] - 0s 540us/step - loss: 0.1050 - acc: 0.9778
Epoch 129/200
135/135 [==============================] - 0s 555us/step - loss: 0.1046 - acc: 0.9778
Epoch 130/200


135/135 [==============================] - 1s 4ms/step - loss: 4.5425 - acc: 0.3333
Epoch 2/200
135/135 [==============================] - 0s 629us/step - loss: 3.6409 - acc: 0.2000
Epoch 3/200
135/135 [==============================] - 0s 585us/step - loss: 2.9267 - acc: 0.0222
Epoch 4/200
135/135 [==============================] - 0s 599us/step - loss: 2.2712 - acc: 0.0444
Epoch 5/200
135/135 [==============================] - 0s 592us/step - loss: 1.8075 - acc: 0.0815
Epoch 6/200
135/135 [==============================] - 0s 577us/step - loss: 1.5180 - acc: 0.0667
Epoch 7/200
135/135 [==============================] - 0s 577us/step - loss: 1.4167 - acc: 0.2000
Epoch 8/200
135/135 [==============================] - 0s 570us/step - loss: 1.3571 - acc: 0.0741
Epoch 9/200
135/135 [==============================] - 0s 577us/step - loss: 1.2777 - acc: 0.1185
Epoch 10/200
135/135 [==============================] - 0s 585us/step - loss: 1.1809 - acc: 0.2593
Epoch 11/200
135/135 [===========

Epoch 84/200
135/135 [==============================] - 0s 548us/step - loss: 0.1914 - acc: 0.9852
Epoch 85/200
135/135 [==============================] - 0s 555us/step - loss: 0.1882 - acc: 0.9926
Epoch 86/200
135/135 [==============================] - 0s 555us/step - loss: 0.1835 - acc: 0.9926
Epoch 87/200
135/135 [==============================] - 0s 577us/step - loss: 0.1806 - acc: 0.9926
Epoch 88/200
135/135 [==============================] - 0s 548us/step - loss: 0.1792 - acc: 0.9926
Epoch 89/200
135/135 [==============================] - 0s 548us/step - loss: 0.1757 - acc: 0.9926
Epoch 90/200
135/135 [==============================] - 0s 570us/step - loss: 0.1737 - acc: 0.9852
Epoch 91/200
135/135 [==============================] - 0s 747us/step - loss: 0.1707 - acc: 0.9852
Epoch 92/200
135/135 [==============================] - 0s 644us/step - loss: 0.1695 - acc: 0.9926
Epoch 93/200
135/135 [==============================] - 0s 577us/step - loss: 0.1655 - acc: 0.9926
Epoch 94/2

135/135 [==============================] - 0s 555us/step - loss: 0.0856 - acc: 0.9926
Epoch 167/200
135/135 [==============================] - 0s 659us/step - loss: 0.0831 - acc: 0.9926
Epoch 168/200
135/135 [==============================] - 0s 673us/step - loss: 0.0816 - acc: 0.9852
Epoch 169/200
135/135 [==============================] - 0s 548us/step - loss: 0.0806 - acc: 0.9926
Epoch 170/200
135/135 [==============================] - 0s 548us/step - loss: 0.0805 - acc: 0.9852
Epoch 171/200
135/135 [==============================] - 0s 555us/step - loss: 0.0807 - acc: 0.9926
Epoch 172/200
135/135 [==============================] - 0s 555us/step - loss: 0.0821 - acc: 0.9852
Epoch 173/200
135/135 [==============================] - 0s 562us/step - loss: 0.0794 - acc: 0.9852
Epoch 174/200
135/135 [==============================] - 0s 548us/step - loss: 0.0799 - acc: 0.9778
Epoch 175/200
135/135 [==============================] - 0s 555us/step - loss: 0.0769 - acc: 0.9926
Epoch 176/200


In [22]:
results.info()

AttributeError: 'numpy.ndarray' object has no attribute 'info'

In [23]:
results

array([ 1.        ,  0.93333334,  1.        ,  1.        ,  1.        ,
        1.        ,  1.        ,  0.93333334,  1.        ,  0.86666667])

In [24]:
estimator